#### RNN

    시퀀스를 구성하는 앞쪽 4개의 숫자가 주어졌을 때, 
    그 다음에 올 숫자를 예측하는 간단한 시퀀스 예측 모델을 만들기  

In [ ]:
import tensorflow as tf  
import numpy as np

#시퀀스 예측 데이터 생성
X = []
Y = []
for i in range(6):
    # [0,1,2,3], [1,2,3,4] 같은 정수의 시퀀스를 만듭니다.  
    lst = list(range(i,i+4))

    # 위에서 구한 시퀀스의 숫자들을 각각 10으로 나눈 다음 저장합니다.
    # SimpleRNN 에 각 타임스텝에 하나씩 숫자가 들어가기 때문에 여기서도 하나씩 분리해서 배열에 저장합니다.
    X.append(list(map(lambda c: [c/10], lst)))

    # 정답에 해당하는 4, 5 등의 정수를 역시 위처럼 10으로 나눠서 저장합니다.  
    Y.append((i+4)/10)

X = np.array(X)
Y = np.array(Y)
for i in range(len(X)):  
    print(X[i], Y[i])


In [ ]:
#시퀀스 예측 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units=10,
                              return_sequences=False,
                              input_shape=[4, 1]),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

    출력을 위한 Dense 레이어가 뒤에 추가돼 있습니다. 여기서 주목해야할 점은 input_shape 입니다.  
    여기서 [4,1]은 각각 timesteps, input_dim을 나타냅니다. 
    timesteps란 순환 신경망이 입력에 대해 계산을 반복하는 횟수이고, input_dim은 입력벡터의 크기를 나타냅니다.

In [ ]:
#네트워크 훈련 및 결과 확인
model.fit(X, Y, epochs=100, verbose=0)  
model.save('simple_rnn_1.h5')  
print(model.predict(X))


    시퀀스 예측 모델은 4 타임스텝에 결쳐 입력을 받고, 마지막에 출력값을 다음 레이어로 반환합니다. 
    우리가 추가한 Dense 레이어에는 별도의 활성화 함수가 없기 때문에 ℎ3은 바로 𝑦3가 됩니다. 
    그리고 이 값과 0.4와의 차이가 𝑚𝑠𝑒, 즉 평균 제곱 오차가 됩니다. 

In [ ]:
print(model.predict(np.array([[[0.6],[0.7],[0.8],[0.9]]])))
print(model.predict(np.array([[[-0.1],[0.0],[0.1],[0.2]]])))


#### LSTM

##### 1. simpleRNN 두개 쌓은 모델

In [ ]:
X = []
Y = []
for i in range(3000):
    # 0~1 사이의 랜덤한 숫자 100 개를 만듭니다.
    lst = np.random.rand(100)
    #print(lst)
    
    # 마킹할 숫자 2개의 인덱스를 뽑습니다.
    idx = np.random.choice(100, 2, replace=False)
    #print(idx)
    
    # 마킹 인덱스가 저장된 원-핫 인코딩 벡터를 만듭니다.
    zeros = np.zeros(100)
    #print(zeros)
    
    zeros[idx] = 1
    #print(zeros)
    
    # 마킹 인덱스와 랜덤한 숫자를 합쳐서 X 에 저장합니다.
    X.append(np.array(list(zip(zeros, lst))))
    #print(X)
    
    # 마킹 인덱스가 1인 값들만 서로 곱해서 Y 에 저장합니다.
    Y.append(np.prod(lst[idx]))
    #print(Y)

print(X[0], Y[0])

In [ ]:
#SimpleRNN 레이어를 사용한 곱셈 문제 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units=30,
                              return_sequences=True,
                              input_shape=[100, 2]),
    tf.keras.layers.SimpleRNN(units=30),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
#SimpleRNN 네트워크 학습
X = np.array(X)
Y = np.array(Y)
# 2560개의 데이터만 학습시킵니다. validation 데이터는 20% 로 지정합니다.
history = model.fit(X[:2560], Y[:2560], epochs=100, validation_split=0.2)

In [ ]:
#SimpleRNN 네트워크 학습 결과 확인
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# Test 데이터에 대한 예측 정확도 확인
model.evaluate(X[2560:], Y[2560:])
prediction = model.predict(X[2560:2560 + 5])  # 5개 테스트 데이터에 대한 예측을 표시합니다.
for i in range(5):
    print(Y[2560 + i], '\t', prediction[i][0], '\tdiff:',
          abs(prediction[i][0] - Y[2560 + i]))

prediction = model.predict(X[2560:])
fail = 0
for i in range(len(prediction)):
    # 오차가 0.04 이상이면 오답입니다.
    if abs(prediction[i][0] - Y[2560 + i]) > 0.04: fail += 1

print('correctness:', (440 - fail) / 440 * 100, '%')

##### 2. LSTM사용 모델

In [ ]:
#LSTM 레이어를 사용한 곱셈 문제 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=30, return_sequences=True, input_shape=[100,
                                                                       2]),
    tf.keras.layers.LSTM(units=30),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
#LSTM 네트워크 학습
X = np.array(X)
Y = np.array(Y)
history = model.fit(X[:2560], Y[:2560], epochs=100, validation_split=0.2)

In [ ]:
# LSTM 네트워크 학습 결과 확인
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
#Test 데이터에 대한 예측 정확도 확인
model.evaluate(X[2560:], Y[2560:])
prediction = model.predict(X[2560:2560 + 5])
for i in range(5):
    print(Y[2560 + i], '\t', prediction[i][0], '\tdiff:',
          abs(prediction[i][0] - Y[2560 + i]))

prediction = model.predict(X[2560:])
cnt = 0
for i in range(len(prediction)):
    if abs(prediction[i][0] - Y[2560 + i]) > 0.04: cnt += 1
print('correctness:', (440 - cnt) / 440 * 100, '%')

#### Bi-LSTM

    시퀀스 또는 시계열 데이터 처리에 특화되어 은닉층에서 과거의 정보를 기억
    순환 신경망  의 구조적 특성상 데이터가 입력 순으로 처리되기 때문에 이전 시점의 정보만 활용할 수 밖에 없다는 단점이 존재
    >  입력 문장을 양방향에서 처리하므로 시퀀스 길이가 길어진다 하더라도 정보손실없이 처리가 가능

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Bidirectional, LSTM

model = Sequential()
model.add(Bidirectional(LSTM(30, return_sequences=True), input_shape=[100, 2]))  
model.add(Bidirectional(LSTM(30)))
model.add(Dense(1))
model.compile(loss='mse', optimizer = 'adam')
model.summary()


In [ ]:
X = np.array(X)   
Y = np.array(Y)
history = model.fit(X, Y, epochs=100, verbose=1, validation_split= 0.2)

In [ ]:
import matplotlib.pyplot as plt  
plt.plot(history.history['loss'], 'b-', label='loss')  
plt.plot(history.history['val_loss'], 'r--', label='val_loss')  
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X[2560:], Y[2560:])
prediction = model.predict(X[2560:2560 + 5])
for i in range(5):
    print(Y[2560 + i], '\t', prediction[i][0], '\tdiff:',
          abs(prediction[i][0] - Y[2560 + i]))

prediction = model.predict(X[2560:])
cnt = 0
for i in range(len(prediction)):
    if abs(prediction[i][0] - Y[2560 + i]) > 0.04:
        cnt += 1
print('correctness:', (440 - cnt) / 440 * 100, '%')

#### 개체명 인식

    문장 내에 포함된 어떤 단어가 인물, 장소, 날짜 등을 의미하는 단어인지 인식하는 것
    개체명 인식은 챗봇에서 문장을 정확하게 해석하기 위해 반드시 해야 하는 전처리 과정

In [ ]:
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split  #데이터 분할
import numpy as np
import warnings

warnings.filterwarnings('ignore')


#학습파일 불러오기
def read_file(file_name):
    sents = []  #
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            # 만약 첫 번째 값이 ; 이 참이며, 두번째 줄의 첫 번째 값도 $ 이 맞으면,
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []  #빈 리스트를 생성
                
            #두 번째 조건, 여기는 그냥 넘어가야 하니 위 조건을 다시 가져와 넘겨준다.
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
                
            #세 번째 조건, 만약 첫 번째 줄이 띄어쓰기라면 this_sent를 sents에 저장
            elif l[0] == '\n':
                sents.append(this_sent)
                
            #위 조건이 다 아니라면 즉, 분석 결과들이라면
             #이 값들을 this_sent에 저장
            else:
                this_sent.append(tuple(l.split()))
    return sents


corpus = read_file('train.txt')

In [ ]:
#문장과 테그를 저장할 빈 리스트 생성
sentences, tags = [], []
#위에서 추출한 corpus 데이터셋 한줄 씩 t에 할당
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    #t에 할당된 corpus의 데이를 또 나눠 w에 할당
    for w in t:
        #그 중 두 번째 텍스트와 마지막 BIO 테그를 동시에 tagged_sentence에 저장
        tagged_sentence.append((w[1], w[3]))
        #각각 따로 저장
        sentence.append(w[1])
        bio_tag.append(w[3])
    #위에서 저장한 객체들을 하나로 묶어준다.
    sentences.append(sentence)
    tags.append(bio_tag)

print("샘플 크기 : \n", len(sentences))
print("2번째 샘플 문장 시퀀스 : \n", sentences[1])
print("2번째 샘플 bio 태그 : \n", tags[1])
print("샘플 문장 시퀀스 최대 길이 : ", max(len(l) for l in sentences))
print("샘플 문장 시퀀스 평균 길이 : ", (sum(map(len, sentences)) / len(sentences)))

In [ ]:
#토크나이저 정의
#미리 인덱싱하지 않은 단어들은 OOV로 지정
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='OOV')
#위의 설정대로 문자를 입력받아 리스트의 형태로 변환
sent_tokenizer.fit_on_texts(sentences)

#테그 정보는 소문자로 변환하지 않도록 셋팅
tag_tokenizer = preprocessing.text.Tokenizer(lower=False)
#위의 설정대로 문자를 입력받아 리스트의 형태로 변환
tag_tokenizer.fit_on_texts(tags)

#문장의 수
vocab_size = len(sent_tokenizer.word_index) + 1
#태그의 수
tag_size = len(tag_tokenizer.word_index) + 1

print("단어 사전 크기 : ", vocab_size)
print("BIO 태그 사전 크기 : ", tag_size)

#학습용 단어 시퀀스 생성 - texts_to_sequences()는 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(x_train[1])
print(y_train[1])

In [ ]:
index_to_word = sent_tokenizer.index_word  #시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word  #시퀀스 인덱스를 NER로 변환하기 위해 사용
#print(index_to_word)
#print(index_to_ner)
index_to_ner[0] = 'PAD'  #첫 번째 값이 의미 없는 'O' 이기에 PAD처리
max_len = 40
print("before : ", x_train[1])
print("before : ", y_train[1])
#아래 함수는 길이가 제각기 다른 데이터 셋에 사용되며, padding을 통해 길이를 일정하게 만들어줍니다.
x_train = preprocessing.sequence.pad_sequences(x_train,
                                               padding='post',
                                               maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train,
                                               padding='post',
                                               maxlen=max_len)
print("after :", x_train[1])
print("after :", y_train[1])

#학습 데이터와 테스트 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x_train,
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=0)

#출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)
print("one-hot / y_train : ", y_train[1])

In [ ]:
#모델 정의 BI-LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(
    Embedding(input_dim=vocab_size,
              output_dim=30,
              input_length=max_len,
              mask_zero=True))

# 양방향 lstm 모델을 정의할 때 정방향, 역방향 lstm 계층에 모든 출력값을 연결해야 하기 때문에 
# return_sequences 인자를 반드시 True
model.add(
    Bidirectional(
        LSTM(200, return_sequences=True, dropout=0.50,
             recurrent_dropout=0.25)))

#또한 Dense 계층이 3차원 텐서를 입력받을 수 있게 확장해야 하므로 TimeDistributed()를 사용해야 합니다.
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.01),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=128, epochs=10)

print("평가결과 : ", model.evaluate(x_test, y_test))

In [ ]:
#시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):
    result = []
    #각 데이터에 대한 분리
    for sequence in sequences:
        temp = []
        #각 데이터에 대한 패딩 처리된 태그별 확률 분리
        for pred in sequence:
            #분리한 값 중 제일 큰 인덱스를 pred_index에 저장
            pred_index = np.argmax(pred)
            #print(pred_index)
            #인덱스에서 다시 NER로 변환하며 앞서 'PAD'처리한 부분을 다시 원래의 'O'로 변환합니다.
            temp.append(index_to_ner[pred_index].replace('PAD', 'O'))
        result.append(temp)
    return result


y_predicted = model.predict(x_test)
#(711,40)->model->(711,40,8(각 NER객체에 대한 확률))
#print(y_predicted)

#비교를 위해서
pred_tags = sequences_to_tag(y_predicted)  #예측된 값에 대한 NER
test_tags = sequences_to_tag(y_test)  #실제 값에 대한 NER

#F1 스코어 계산 과정
from seqeval.metrics import f1_score, classification_report

print(classification_report(test_tags, pred_tags))
print("F1-score : {:.1%}".format(f1_score(test_tags, pred_tags)))

In [ ]:
#새로운 유형의 문장 NER예측
word_to_index = sent_tokenizer.word_index  #각 단어들을 인덱스로 전환
new_sentence = "삼성전자 주가는 오늘 스마트폰 출시로 올라갔다.".split()
new_x = []
for w in new_sentence:
    try:
        #get(key,'디폴트')는 key에 대한 value를 반환, 이때 존재하지 않는키면 KeyError를 불러일으킴
        #이때 디폴트값이 정해진 경우라면 그 값을 반환  #여기서 1이란 값은 'OOV' 입니다.
        new_x.append(word_to_index.get(w, 1))
        
    except KeyError:
        new_x.append(word_to_index['OOV'])

#위의 삼성전자 예시를 쪼개서 인덱싱한 결과를 보여줌
print("새로운 유형의 시퀀스 : ", new_x)

#패딩 처리
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x],
                                                       padding='post',
                                                       value=0,
                                                       maxlen=max_len)

#NER 예측
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis=1)

#출력 결과 표시
print("{:10} {:5}".format("단어", "예측된 NER"))
print("-" * 50)